# Problem Set 04: HMMs

In this problem set you will implement value iteration in a non-deterministic MDP, and HMM Filtering.


0. [Credit for Contributors (required)](#contributors)

      
1. [HMM Filtering](#problem1)
    1. [Implement `compute_casino_table` for fair dice estimation (40 points)](#compute_casino_table)
    2. [Dendrochronology Example (20 points)](#viterbi)
    
**60 points** total for Problem Set 4



## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

Worked with Anoop on finding an ambiguity in how to interpret the transition function of the Tree Rings Problem. Worked with Anoop and Shashank to figure out how to correctly derive the sequence of states for decoding (Dice problem and tree rings problem) once calculations were complete. I found it difficult to understand smoothing in the slides. However, I found a useful youtube video that helped me realize how to do smoothing for the dice problem, carrying over b from each backward recursion. https://www.youtube.com/watch?v=A-gTxz-CrsE | This video was also very helpful to understand decoding for the dice decoding and the tree ring problem. https://www.youtube.com/watch?v=xejm-z3sbWA

***Note that I did both additional credit problems: smoothing and decoding for the dishonest casino dice problem! The code and solutions are below the filtering part.***

In [3]:
# Useful imports

from __future__ import division

%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
from nose.tools import ok_, assert_equal, assert_almost_equal

## <a name="problem1"/></a> Problem 1 : HMM  Filtering (60 points)

In this problem you will perform filtering on an HMM using the forward algorithm (as explained in class).

Remember that the filtering problem answers the question

\begin{equation}
Pr(S_t | o_{1:t})
\end{equation}

that is, what is the most likely state at time $t$ after reading $t$ observations.

You will implement filtering and demonstrate it for the "dishonest casino" example HMM that was introduced in class. The parameters for the HMM are shown in the following image (and also in the lecture notes).

<img src='hmm_casino.png'/>

Also, the prior for the states is

\begin{align}
    &Pr(S_0 = Fair) &= 0.8\\
    &Pr(S_0 = Loaded) &= 0.2
\end{align}

that is, the probability that the initial state is "Fair" is 0.8, and the probability that the initial state is "Loaded" is 0.2.

Implement filtering for this HMM for the following observation sequence:

`['1','2','4','6','6','6','3','6']`

Your code should produce a table like the following one (from 0 measurements to the totality of the measurements provided).


```
Filtering for the dishonest casino HMM:
  with 0 measurement(s): [], P(s at t=0) is: [Fair: 0.8000,  Loaded: 0.2000]
  with 1 measurement(s): ['1'], P(s at t=1) is: [Fair: 0.8480,  Loaded: 0.1520]
  with 2 measurement(s): ['1', '2'], P(s at t=2) is: [Fair: 0.8789,  Loaded: 0.1211]
  with 3 measurement(s): ['1', '2', '4'], P(s at t=3) is: [Fair: 0.8981,  Loaded: 0.1019]
  with 4 measurement(s): ['1', '2', '4', '6'], P(s at t=4) is: [Fair: 0.6688,  Loaded: 0.3312]
  ...
```

Note that this is exactly the example given in class and that you have the results in the slides.

<div class="alert alert-info">
Implement HMM filtering and use it in the "dishonest casino" example HMM to generate a table like the one above.
</div>


**Note**: you could implement `smoothing` and `decoding` on the same HMM model and observation data for **additional credit** (but you don't have to if you don't want). 

If you do implement smoothing and decoding, please make it explicit in a separate cell below for our graders to see.

In [4]:
# You might find the following examples on how to use numpy for matrices and vectors useful

# Define a 2x3 Matrix
M=np.array([[3.5, 2.1, 1.3],
            [0.1, 0.2, 0.3]])
print("Matrix M is:")
print(M)

print("Element (0,1) of M is: ", M[0,1])
print("Shape (dimension) of M is: ", M.shape)

# Define 1x2 vector
v =  np.array([1, 3])
print("Vector v is:")
print(v)

# Matrix multiplication r = v*M
r = v.dot(M)
print("Matrix multiplication v * M = ", r)

# Element wise multiplication
a = np.array([2, 4])
print("Element wise v * a = %s * %s = "%(v,a), v*a)

# Sum of elements of v
print("Sum of elements of v (%s) is: %s" %(v, np.sum(v)))

# Vector slicing
b = np.array([2,4,6,8,10,12,14,16])
print("Vector b is: ", b)
print("First 3 elements of b are: ", b[:3])
print("Elements from the second until the end are: ", b[1:])
print("Elements from 4th to 7th are: ", b[3:7])

Matrix M is:
[[3.5 2.1 1.3]
 [0.1 0.2 0.3]]
Element (0,1) of M is:  2.1
Shape (dimension) of M is:  (2, 3)
Vector v is:
[1 3]
Matrix multiplication v * M =  [3.8 2.7 2.2]
Element wise v * a = [1 3] * [2 4] =  [ 2 12]
Sum of elements of v ([1 3]) is: 4
Vector b is:  [ 2  4  6  8 10 12 14 16]
First 3 elements of b are:  [2 4 6]
Elements from the second until the end are:  [ 4  6  8 10 12 14 16]
Elements from 4th to 7th are:  [ 8 10 12 14]


 ###  <a name="compute_casino_table"></a>Implement HMM Filter

Write your code as a function:

```
def compute_casino_table(meas)
```

that takes as input the observation sequence, and prints the table discussed above.


In [5]:
def compute_casino_table(meas):
    # Example observation sequence
    # meas = ['1','2','4','6','6','6','3','6']
    meas = [int(i) for i in meas]
    P_F = 0.8
    P_L = 0.2
    Init = np.array([P_F,P_L]) #starting probabilities (fair, loaded)
    T = np.array([[0.95,0.05],[0.05,0.95]]) #Transition Matrix (changes from fair to loaded 1/20 times)
    M = np.array([[1/6,1/6,1/6,1/6,1/6,1/6],[1/10,1/10,1/10,1/10,1/10,1/2]]) #observation function, top row is fair, bottom is loaded
    prob_filtered = np.copy(Init)
    fair_vector = np.copy(Init[0])
    loaded_vector = np.copy(Init[1])
    for i in range(len(meas)):
        forward_fair = M[0,meas[i]-1]*(T[0,0]*prob_filtered[0]+T[0,1]*prob_filtered[1])
        forward_loaded = M[1,meas[i]-1]*(T[1,0]*prob_filtered[0]+T[1,1]*prob_filtered[1])
        bayes = 1/(forward_fair+forward_loaded)
        fair = forward_fair*bayes
        loaded = forward_loaded*bayes
        prob_filtered = np.array([fair,loaded])
        fair_vector = np.append(fair_vector,fair)
        loaded_vector = np.append(loaded_vector,loaded)
        
    print("Filtering for the dishonest casino HMM:")
    for i in range(len(meas)+1):
        if i == 0:
            output_str = f"with {i} measurement(s): [], P(s at t={i}) is: [Fair: {fair_vector[i]:.4f}, Loaded: {loaded_vector[i]:.4f}]"    
        else:
            output_str = f"with {i} measurement(s): {meas[:i]}, P(s at t={i}) is: [Fair: {fair_vector[i]:.4f}, Loaded: {loaded_vector[i]:.4f}]"        
        print(output_str)
    #return fair_vector, loaded_vector
    

In [7]:
# Don't modify this cell
meas = ['1','2','4','6','6','6','3','6']
compute_casino_table(meas)

Filtering for the dishonest casino HMM:
with 0 measurement(s): [], P(s at t=0) is: [Fair: 0.8000, Loaded: 0.2000]
with 1 measurement(s): [1], P(s at t=1) is: [Fair: 0.8480, Loaded: 0.1520]
with 2 measurement(s): [1, 2], P(s at t=2) is: [Fair: 0.8789, Loaded: 0.1211]
with 3 measurement(s): [1, 2, 4], P(s at t=3) is: [Fair: 0.8981, Loaded: 0.1019]
with 4 measurement(s): [1, 2, 4, 6], P(s at t=4) is: [Fair: 0.6688, Loaded: 0.3312]
with 5 measurement(s): [1, 2, 4, 6, 6], P(s at t=5) is: [Fair: 0.3843, Loaded: 0.6157]
with 6 measurement(s): [1, 2, 4, 6, 6, 6], P(s at t=6) is: [Fair: 0.1793, Loaded: 0.8207]
with 7 measurement(s): [1, 2, 4, 6, 6, 6, 3], P(s at t=7) is: [Fair: 0.3088, Loaded: 0.6912]
with 8 measurement(s): [1, 2, 4, 6, 6, 6, 3, 6], P(s at t=8) is: [Fair: 0.1399, Loaded: 0.8601]


### Smoothing (additional credit)

In [9]:
def compute_casino_table_with_smoothing(meas):
    # Example observation sequence
    # meas = ['1','2','4','6','6','6','3','6']
    meas = [int(i) for i in meas]
    P_F = 0.8
    P_L = 0.2
    Init = np.array([P_F,P_L]) #starting probabilities (fair, loaded)
    T = np.array([[0.95,0.05],[0.05,0.95]]) #Transition Matrix (changes from fair to loaded 1/20 times)
    M = np.array([[1/6,1/6,1/6,1/6,1/6,1/6],[1/10,1/10,1/10,1/10,1/10,1/2]]) #observation function, top row is fair, bottom is loaded
    prob_filtered = np.copy(Init)
    fair_vector = np.copy(Init[0])
    loaded_vector = np.copy(Init[1])
    for i in range(len(meas)):
        forward_fair = M[0,meas[i]-1]*(T[0,0]*prob_filtered[0]+T[0,1]*prob_filtered[1])
        forward_loaded = M[1,meas[i]-1]*(T[1,0]*prob_filtered[0]+T[1,1]*prob_filtered[1])
        bayes = 1/(forward_fair+forward_loaded)
        fair = forward_fair*bayes
        loaded = forward_loaded*bayes
        prob_filtered = np.array([fair,loaded])
        fair_vector = np.append(fair_vector,fair)
        loaded_vector = np.append(loaded_vector,loaded)
        
    #pdb.set_trace()
    smooth_fair_vector = []
    smooth_loaded_vector = []
    for i in range(len(fair_vector) - 1, -1, -1):
        if i == 8:
            # init recursive, this is the same as filter b/c no observation at last time step
            smooth_fair = fair_vector[i]
            smooth_loaded = loaded_vector[i]
            smooth_fair_vector = np.append(smooth_fair_vector,smooth_fair)
            smooth_loaded_vector = np.append(smooth_loaded_vector,smooth_loaded)
        elif i==7:
            # do this to create fair_check, which will be carried over to account for other observations
            #fair_check = fair_vector[i]*(M[0,meas[i]-1]*T[0,0] + M[1,meas[i]-1]*T[1,0])
            #loaded_check = loaded_vector[i]*(M[0,meas[i]-1]*T[0,1] + M[1,meas[i]-1]*T[1,1])
            fair_check = M[0,meas[i]-1]*T[0,0] + M[1,meas[i]-1]*T[1,0]
            loaded_check = M[0,meas[i]-1]*T[0,1] + M[1,meas[i]-1]*T[1,1]
            #bayes = fair_check + loaded_check
            #smooth_fair = fair_check/bayes
            #smooth_loaded = loaded_check/bayes
            bayes = fair_vector[i]*fair_check + loaded_vector[i]*loaded_check
            smooth_fair = (fair_vector[i]*fair_check)/bayes
            smooth_loaded = (loaded_vector[i]*loaded_check)/bayes
            
            smooth_fair_vector = np.append(smooth_fair_vector,smooth_fair)
            smooth_loaded_vector = np.append(smooth_loaded_vector,smooth_loaded)
        #elif i==6:
            #fair_check = fair_vector[i]*(M[0,meas[i+1]-1]*M[0,meas[i]-1]*T[0,0] + M[1,meas[i+1]-1]*M[1,meas[i]-1]*T[1,0])
            #loaded_check = loaded_vector[i]*(M[0,meas[i+1]-1]*M[0,meas[i]-1]*T[0,1] + M[1,meas[i+1]-1]*M[1,meas[i]-1]*T[1,1])
            #fair_check_2 = fair_check*M[0,meas[i]-1]*T[0,0] + loaded_check*M[1,meas[i]-1]*T[1,0]
            #loaded_check_2 = fair_check*M[0,meas[i]-1]*T[0,1] + loaded_check*M[1,meas[i]-1]*T[1,1]
            #bayes = fair_check + loaded_check
            #smooth_fair = fair_check/bayes
            #smooth_loaded = loaded_check/bayes
            #bayes = fair_vector[i]*fair_check_2 + loaded_vector[i]*loaded_check_2
            #smooth_fair = (fair_vector[i]*fair_check_2)/bayes
            #smooth_loaded = (loaded_vector[i]*loaded_check_2)/bayes
            #smooth_fair_vector = np.append(smooth_fair_vector,smooth_fair)
            #smooth_loaded_vector = np.append(smooth_loaded_vector,smooth_loaded)
        else:
            # this loops through the rest of it, carrying through fair_check and adding in the k+1 observation
            fair_check_2 = fair_check*M[0,meas[i]-1]*T[0,0] + loaded_check*M[1,meas[i]-1]*T[1,0]
            loaded_check_2 = fair_check*M[0,meas[i]-1]*T[0,1] + loaded_check*M[1,meas[i]-1]*T[1,1]
            fair_check_forward = fair_vector[i]*fair_check_2
            loaded_check_forward = loaded_vector[i]*loaded_check_2
            bayes = fair_check_forward + loaded_check_forward
            smooth_fair = fair_check_forward/bayes
            smooth_loaded = loaded_check_forward/bayes
            smooth_fair_vector = np.append(smooth_fair_vector,smooth_fair)
            smooth_loaded_vector = np.append(smooth_loaded_vector,smooth_loaded)
            fair_check = fair_check_2
            loaded_check = loaded_check_2
    
    ans1 = smooth_fair_vector[::-1]
    ans2 = smooth_loaded_vector[::-1]
    
    print("Smoothing for the dishonest casino HMM:")
    for i in range(len(meas)+1):
        if i == 0:
            output_str = f"P(s at t={i}) is: [Fair: {ans1[i]:.4f}, Loaded: {ans2[i]:.4f}]"    
        else:
            output_str = f"P(s at t={i}) is: [Fair: {ans1[i]:.4f}, Loaded: {ans2[i]:.4f}]"        
        print(output_str)
    #return fair_vector, loaded_vector         
    
    return ans1, ans2

In [10]:
meas = ['1','2','4','6','6','6','3','6']
smooth_fair_vector, smooth_loaded_vector = compute_casino_table_with_smoothing(meas)

Smoothing for the dishonest casino HMM:
P(s at t=0) is: [Fair: 0.7382, Loaded: 0.2618]
P(s at t=1) is: [Fair: 0.6940, Loaded: 0.3060]
P(s at t=2) is: [Fair: 0.6116, Loaded: 0.3884]
P(s at t=3) is: [Fair: 0.4679, Loaded: 0.5321]
P(s at t=4) is: [Fair: 0.2229, Loaded: 0.7771]
P(s at t=5) is: [Fair: 0.1444, Loaded: 0.8556]
P(s at t=6) is: [Fair: 0.1265, Loaded: 0.8735]
P(s at t=7) is: [Fair: 0.1449, Loaded: 0.8551]
P(s at t=8) is: [Fair: 0.1399, Loaded: 0.8601]


### Decoding (additional credit)

In [13]:
def compute_casino_table_with_decoding(meas):
    # Example observation sequence
    # meas = ['1','2','4','6','6','6','3','6']
    meas = [int(i) for i in meas]
    P_F = 0.8
    P_L = 0.2
    Init = np.array([P_F,P_L]) #starting probabilities (fair, loaded)
    T = np.array([[0.95,0.05],[0.05,0.95]])
    #T = np.array([[0.5,0.5],[0.5,0.5]])#Transition Matrix (changes from fair to loaded 1/20 times)
    M = np.array([[1/6,1/6,1/6,1/6,1/6,1/6],[1/10,1/10,1/10,1/10,1/10,1/2]]) #observation function, top row is fair, bottom is loaded
    prob_filtered = np.copy(Init)
    fair_vector = np.copy(Init[0])
    loaded_vector = np.copy(Init[1])
    for k in range(len(meas)):
        forward_fair = M[0,meas[k]-1]*(T[0,0]*prob_filtered[0]+T[0,1]*prob_filtered[1])
        forward_loaded = M[1,meas[k]-1]*(T[1,0]*prob_filtered[0]+T[1,1]*prob_filtered[1])
        bayes = 1/(forward_fair+forward_loaded)
        fair = forward_fair*bayes
        loaded = forward_loaded*bayes
        prob_filtered = np.array([fair,loaded])
        fair_vector = np.append(fair_vector,fair)
        loaded_vector = np.append(loaded_vector,loaded)
        
    
    for j in range(8):
    #for i in range(len(meas)):
        viterbi = np.copy(Init)
        viterbi_fair = np.copy(Init[0])
        viterbi_loaded = np.copy(Init[1])
        loaded_type = []
        fair_type = []
        viterbi_out = []
        #pdb.set_trace()
        for i in range(j+1):
            #pdb.set_trace()
            #Fair
            fair1 = M[0,meas[i]-1]*T[0,0]*viterbi[0] #P(F/F)
            fair2 = M[0,meas[i]-1]*T[0,1]*viterbi[1] #P(F/L)
            fair = max(fair1,fair2)
            if fair1 > fair2:
                fair_type.append('F/F')
            else:
                fair_type.append('F/L')
            
            #Loaded
            loaded1 = M[1,meas[i]-1]*T[1,0]*viterbi[0] #P(L/F)
            loaded2 = M[1,meas[i]-1]*T[1,1]*viterbi[1] #P(L/L)
            loaded = max(loaded1,loaded2)
            if loaded1 > loaded2:
                loaded_type.append('L/F')
            else:
                loaded_type.append('L/L')
            
            
            viterbi = np.array([fair,loaded])
            viterbi_fair = np.append(viterbi_fair,fair)
            viterbi_loaded = np.append(viterbi_loaded,loaded)
             
            
        if fair > loaded:
            viterbi_last = 'fair'
        else:
            viterbi_last = 'loaded'
        #pdb.set_trace()              
        sequence = []
        loaded_check = loaded_type[::-1]
        fair_check = fair_type[::-1]
        viterbi_loaded_reverse = viterbi_loaded[::-1]
        viterbi_fair_reverse = viterbi_fair[::-1]
        #pdb.set_trace()
        for i in range(len(loaded_check)):
            if i == 0:
                if viterbi_last == 'loaded':
                    sequence.append('loaded')
                elif viterbi_last == 'fair':
                    sequence.append('fair')
                else:
                    sequence.append('equal')
            else:
                if sequence[i-1] == 'loaded': #if loaded only two options can be L/F or L/L (had to come from the loaded calculation)
                    if loaded_check[i-1] == 'L/L':
                        sequence.append('loaded')
                    else:
                        sequence.append('fair')
                else: #if fair only two options can be F/F or F/L
                    if fair_check[i-1] == 'F/F':
                        sequence.append('fair')
                    else:
                        sequence.append('loaded')           
        #pdb.set_trace()                 
        sequence = sequence[::-1]        
        output_str = f"Most likely state sequence for t = 1:{j+1} is: {sequence}"
        print(output_str)        
    return sequence

In [14]:
# Decoding Check
meas = ['1','2','4','6','6','6','3','6']
viterbi_out = compute_casino_table_with_decoding(meas)

Most likely state sequence for t = 1:1 is: ['fair']
Most likely state sequence for t = 1:2 is: ['fair', 'fair']
Most likely state sequence for t = 1:3 is: ['fair', 'fair', 'fair']
Most likely state sequence for t = 1:4 is: ['fair', 'fair', 'fair', 'fair']
Most likely state sequence for t = 1:5 is: ['fair', 'fair', 'fair', 'fair', 'fair']
Most likely state sequence for t = 1:6 is: ['loaded', 'loaded', 'loaded', 'loaded', 'loaded', 'loaded']
Most likely state sequence for t = 1:7 is: ['fair', 'fair', 'fair', 'fair', 'fair', 'fair', 'fair']
Most likely state sequence for t = 1:8 is: ['loaded', 'loaded', 'loaded', 'loaded', 'loaded', 'loaded', 'loaded', 'loaded']


 ###  <a name="viterbi"></a>Dendrochronology Example
 
Dendrochronology (or tree-ring dating) is the scientific method of dating tree rings (also called growth rings) to the exact year they were formed in order to analyze atmospheric conditions during different periods in history. [Wikipedia]

<img src="tree-rings-diagram.jpg" style="width:40%;"/>
<p style="text-align:center">Image source: NASA </p>

Let’s say we are interested in estimating the average annual temperature at a particular location
on Earth. However, the time period that we are interested in is in distant past, with no direct
temperature data available. Prior research indicates correlation between size of tree rings and temperature, so we decide to use this as our evidence.

To simplify the problem, we’ll only consider two temperature states, “Hot” and “Cold”. Suppose
that we are told the probability of hot year followed by another hot year is 0.7 and the probability
that a cold year is followed by another cold year is 0.6. Assume that the initial state distribution, π, is given. For observations, we’ll only consider three different tree ring sizes, small, medium, and large, or S, M , and L, respectively. The probabilistic relationship between the annual temperature and the ring sizes is the observation matrix. The state transition matrix, initial distribution, and observation matrix are provided below:

<img src="temp_example.png"/>

For example, the probability of observing a “Large” tree ring size given that the state is “Hot” is 0.5.


From a particular four-year period of interest, we observe the series of tree rings, **[S, M, S, L]**. 
Assume that $\pi$ is for $s_0$ and the observations are for $s_1, \ldots, s_4$ i.e $o_1 = $ **S** and so on. 

With this observation sequence, find the most likely temperature sequence $[s_1, s_2, s_3, s_4]$ using the **Viterbi
algorithm**. 

Please show how you found the most likely sequence.


<div class="alert alert-info">
Write or upload your answer in the cell below this one.
</div>

![title](Tree_Rings_2.jpg)